روی داده های برق داده شده تمام کارهای مورد نیار را انجام داده و در انتها داده با فیچر های زیر را نیز پیش بینی کنید

"CO2Intensity"=491.32, "SystemLoadEA"=4241.05, "Day"=10, "ForecastWindProduction"=54.10, , "SMPEA"=49.56, "ORKTemperature"=9.0, "ORKWindspeed"=14.8, "ActualWindProduction"=54.0, "Month"=12, "SystemLoadEP2"=4426.84]

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [59]:
df = pd.read_csv("electricity.csv",parse_dates= ["DateTime"] , na_values = ['None' , '?'] )

In [12]:
df.head(5)

,DateTime,Holiday,HolidayFlag,DayOfWeek,WeekOfYear,Day,Month,Year,PeriodOfDay,ForecastWindProduction,SystemLoadEA,SMPEA,ORKTemperature,ORKWindspeed,CO2Intensity,ActualWindProduction,SystemLoadEP2,SMPEP2
0,2011-01-11 00:00:00,NaN,0,1,44,1,11,2011,0,315.31,3388.77,49.26,6.0,9.3,600.71,356.0,3159.60,54.32
1,2011-01-11 00:30:00,NaN,0,1,44,1,11,2011,1,321.80,3196.66,49.26,6.0,11.1,605.42,317.0,2973.01,54.23
2,2011-01-11 01:00:00,NaN,0,1,44,1,11,2011,2,328.57,3060.71,49.10,5.0,11.1,589.97,311.0,2834.00,54.23
3,2011-01-11 01:30:00,NaN,0,1,44,1,11,2011,3,335.60,2945.56,48.04,6.0,9.3,585.94,313.0,2725.99,53.47
4,2011-01-11 02:00:00,NaN,0,1,44,1,11,2011,4,342.90,2849.34,33.75,6.0,11.1,571.52,346.0,2655.64,39.87


In [13]:
df.isna().sum()

DateTime                      0
Holiday                   36478
HolidayFlag                   0
DayOfWeek                     0
WeekOfYear                    0
Day                           0
Month                         0
Year                          0
PeriodOfDay                   0
ForecastWindProduction        5
SystemLoadEA                  2
SMPEA                         2
ORKTemperature              295
ORKWindspeed                299
CO2Intensity                  7
ActualWindProduction          5
SystemLoadEP2                 2
SMPEP2                        2
dtype: int64

In [14]:
df["Holiday"].unique()

array([nan, 'Christmas Eve', 'Christmas', "St Stephen's Day",
       "New Year's Eve", "New Year's Day", "St Patrick's Day",
       'Good Friday', 'Holy Saturday', 'Easter', 'Easter Monday',
       'May Day', 'June Bank Holiday', 'August Bank Holiday',
       'October Bank Holiday'], dtype=object)

### Encoding holiday labels

In [28]:
ordered_labels = df.groupby("Holiday").mean("SMPEP2")["SMPEP2"].sort_values().index.to_list()

In [60]:
order_labels_map = {0 : 0}
for indx , label in enumerate(ordered_labels,start=1):
    order_labels_map[label] = indx

order_labels_map

{0: 0,
 "St Stephen's Day": 1,
 "New Year's Eve": 2,
 'Christmas': 3,
 "New Year's Day": 4,
 'Christmas Eve': 5,
 'August Bank Holiday': 6,
 'Easter Monday': 7,
 'October Bank Holiday': 8,
 'Holy Saturday': 9,
 'June Bank Holiday': 10,
 'May Day': 11,
 'Good Friday': 12,
 "St Patrick's Day": 13,
 'Easter': 14}

In [61]:
df["Holiday"].fillna(0 ,inplace= True)
df["Holiday"] = df["Holiday"].map(order_labels_map)
# # df["Holiday"].map(order_labels_map, na_action="ignore")
df["Holiday"]

0        0
1        0
2        0
3        0
4        0
        ..
38009    2
38010    2
38011    2
38012    2
38013    2
Name: Holiday, Length: 38014, dtype: int64

In [66]:
df["SMPEP2"].nunique() # So it cannot be considered categorical

7813

In [69]:
# datetime is redundant as we already have year,month... columns
df.drop(["DateTime"],axis=1 , inplace= True)

In [82]:
df_dropped_none = df.dropna()

In [91]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error , mean_absolute_error ,r2_score
from sklearn.model_selection import cross_validate, GridSearchCV ,train_test_split

In [92]:
X = df_dropped_none.drop(["SMPEP2"],axis=1)
y = df_dropped_none["SMPEP2"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [99]:
pipe = Pipeline([('scaler', (MinMaxScaler())), ('knn_reg', KNeighborsRegressor())])

grid_space = {"knn_reg__n_neighbors" : [5,10,15,20,30,100,150] ,
"knn_reg__weights" : ['uniform', 'distance'] ,
 "knn_reg__leaf_size":[3,5,10,20,20,50] }
gs_reg = GridSearchCV(pipe , param_grid = grid_space,cv=5 ) # , scoring=mean_absolute_error 

In [100]:
gs_reg.fit(X_train,y_train)

KeyboardInterrupt: 

به صورت واترمارک کنید و خروجی را ذخیره کنید logo.pngروی عکس یکی از دوستان یا خانواده